In [3]:
from pathlib import Path


current_path = Path().resolve()
print(current_path)

/workspace/upstageailab-langchain-pjt-langchain_8/notebooks/minju


In [ ]:
import json
import os
from pathlib import Path

from langchain_core.documents import Document
from langchain_upstage.document_parse import UpstageDocumentParseLoader

In [2]:
assert os.getenv("UPSTAGE_API_KEY"), "환경변수 UPSTAGE_API_KEY가 필요합니다."

In [3]:
FILEPATH = Path(
    "/workspace/upstageailab-langchain-pjt-langchain_8/data/Part 2. RAG with LLamaIndex.pdf"
)  # 예: Path("data/source_pdfs/sample.pdf")
assert FILEPATH.exists(), f"파일이 존재하지 않습니다: {FILEPATH}"

In [4]:
# --- 2) Loader로 문서 파싱 (페이지 단위 + Markdown) ---
# coordinates 옵션은 라이브러리 버전에 따라 기본 포함됩니다.
# 별도 옵션 없이도 metadata에 'coordinates'가 있는지 확인해 보세요.
loader = UpstageDocumentParseLoader(
    FILEPATH,
    api_key=os.environ["UPSTAGE_API_KEY"],
    split="page",  # 페이지 단위로 Document 생성
    output_format="markdown",  # 본문은 markdown 텍스트
)

docs: list[Document] = loader.load()
print(f"로드된 Document 개수: {len(docs)}")

로드된 Document 개수: 34


In [ ]:
PREVIEW_LEN = 400

for i, d in enumerate(docs[:2], start=1):
    print("=" * 80)
    print(f"[샘플 Document #{i}]")
    print("- page_content (앞 400자) -")
    print(d.page_content[:400].strip().replace("\n", " ") + (" ..." if len(d.page_content) > PREVIEW_LEN else ""))
    print("- metadata keys -")
    print(list(d.metadata.keys()))
    print("- metadata preview -")
    # 좌표가 있으면 preview만; 너무 크면 전부 출력하지 말고 일부만 보자
    meta = dict(d.metadata)
    if "coordinates" in meta:
        coords = meta.pop("coordinates")
        print(
            "  coordinates: (길이/타입만 확인)",
            type(coords),
            f"len={len(coords) if hasattr(coords, '__len__') else 'N/A'}",
        )
    print(json.dumps(meta, ensure_ascii=False, indent=2, default=str))

[샘플 Document #1]
- page_content (앞 400자) -
# RAG를 활용한 완성도 높은 LLM 서비스 구축 # With Langchain & LLamaIndex
- metadata keys -
['page', 'coordinates']
- metadata preview -
  coordinates: (길이/타입만 확인) <class 'list'> len=2
{'page': 1}
[샘플 Document #2]
- page_content (앞 400자) -
PART # RAG with LlamaIndex
- metadata keys -
['page', 'coordinates']
- metadata preview -
  coordinates: (길이/타입만 확인) <class 'list'> len=2
{'page': 2}


In [ ]:
if docs and "coordinates" in docs[0].metadata:
    coords0 = docs[0].metadata["coordinates"]
    print("\n[coordinates 예시 1~3개만 보기]")
    if isinstance(coords0, list):
        for j, c in enumerate(coords0[:3], start=1):
            print(f"  - item#{j}:")
            print(json.dumps(c, ensure_ascii=False, indent=2, default=str))
    else:
        print(json.dumps(coords0, ensure_ascii=False, indent=2, default=str))


[coordinates 예시 1~3개만 보기]
  - item#1:
[{'x': 0.0366, 'y': 0.4041},
 {'x': 0.4999, 'y': 0.4041},
 {'x': 0.4999, 'y': 0.4627},
 {'x': 0.0366, 'y': 0.4627}]
  - item#2:
[{'x': 0.0369, 'y': 0.5311},
 {'x': 0.4078, 'y': 0.5311},
 {'x': 0.4078, 'y': 0.587},
 {'x': 0.0369, 'y': 0.587}]
